In [1]:
import os
import pickle
import numpy as np
import pandas as pd
from sklearn import tree

In [2]:
def pickle_read(name):
    file_Name = name
    fileObject = open(file_Name, 'rb')
    var = pickle.load(fileObject)
    fileObject.close()
    return var

In [3]:
def get_ct(ct, s_norm, feature_names, class_names, precision=1):
    """
    To extract rules from classification tree, i.e., module application condition
    
    ct: the object of trained model of classification tree
    s_norm: [int] the absolute value of reservoir storage that is used for normalization
    feature_names: [list of str] names of used variables, e.g., [Inflow, Storage, PDSI, DOY]
    class_names: the class names to be classified, i.e., modules 
                    in this case, if not use specified class names, one can simply use ct.classes_.tolist()
    precision: default decimal precision to output each variable
    
    """
    tree_ = ct.tree_
    feature_name = [feature_names[i] if i != tree._tree.TREE_UNDEFINED else "undefined!" for i in tree_.feature]

    paths = []
    path = []
    
    def recurse(node, path, paths):
        if tree_.feature[node] != tree._tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            
            # specifiy decimal precisions for different variables
            if name in ['Inflow', 'Storage']:
                threshold = threshold * s_norm
                precision = 1    # keep 1 decimal
            if name == 'DOY':
                threshold = int(threshold)    # round down DOY to integer
                precision = 1    # randomly assign a value to avoid the error "precision referenced before assignment"
            if name == 'PDSI':
                precision = 2    # keep 3 decimal
            
            p1, p2 = list(path), list(path)
            p1 += [f"({name} <= {np.round(threshold, precision)})"]
            recurse(tree_.children_left[node], p1, paths)
            p2 += [f"({name} > {np.round(threshold, precision)})"]
            recurse(tree_.children_right[node], p2, paths)
        else:
            path += [(tree_.value[node], tree_.n_node_samples[node])]
            paths += [path]
            
    recurse(0, path, paths)

#     # This allows you to sort the printed lines by samples of each decision tree node
#     # sort by samples count
#     samples_count = [p[-1][1] for p in paths]
#     ii = list(np.argsort(samples_count))
#     paths = [paths[i] for i in reversed(ii)]
    
    rules = []
    for path in paths:
        rule = "if "
        for p in path[:-1]:
            if rule != "if ":
                rule += " and "
            rule += str(p)
        rule += " then "
        classes = path[-1][0][0]
        l = np.argmax(classes)
#         # this allows to print probability for each rule
#             rule += f"module: {class_names[l]} (proba: {np.round(100.0*classes[l]/np.sum(classes),2)}%)"
        rule += f"module: {class_names[l]}"    
#         # this allows to print number of samples for each rule
#         rule += f" | based on {path[-1][1]:,} samples"
        rules += [rule]
        
    return rules
    

In [9]:
"""
Example to print a classification tree
    in the GDROM, the module application condition is a single classification tree object. 
"""
features = ['Inflow', 'Storage', 'PDSI', 'DOY']
file = 'path_to_your_DecisinTreeClassifier'
smax = 562474
ct = pickle_read(file)
rules = get_ct(ct, smax, features, class_names=ct.classes_.tolist())
with open('output_file.txt', 'w') as f:
    for item in rules:
        f.write(f"{item}\n")
        

In [19]:
def get_rt(rt, s_norm, feature_names, class_names=None, precision=1):
    """
    To extract rules from regression tree
        this function can be used to print module rules, or to print operation rules for single-module reservoirs
    """
    
    tree_ = rt.tree_
    feature_name = [feature_names[i] if i != tree._tree.TREE_UNDEFINED else "undefined!" for i in tree_.feature]

    paths = []
    path = []
    
    def recurse(node, path, paths):
        if tree_.feature[node] != tree._tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            if name in ['Inflow', 'Storage']:
                threshold = threshold * s_norm
            p1, p2 = list(path), list(path)
            p1 += [f"({name} <= {np.round(threshold, precision)})"]
            recurse(tree_.children_left[node], p1, paths)
            p2 += [f"({name} > {np.round(threshold, precision)})"]
            recurse(tree_.children_right[node], p2, paths)
        else:
            path += [(tree_.value[node], tree_.n_node_samples[node])]
            paths += [path]
            
    recurse(0, path, paths)

#     # sort by samples count
#     samples_count = [p[-1][1] for p in paths]
#     ii = list(np.argsort(samples_count))
#     paths = [paths[i] for i in reversed(ii)]
    
    rules = []
    for path in paths:
        rule = "if "
        for p in path[:-1]:
            if rule != "if ":
                rule += " and "
            rule += str(p)
        rule += " then "
        if class_names is None:
            rule += "Release: "+str(np.round(path[-1][0][0][0]*s_norm, precision))
        else:
            raise('please set class_names as None for extracting regression trees')
        rules += [rule]
        
    return rules

In [20]:
"""
Example to print a regression tree
    in the GDROM, a single module is a regression tree object. 
    Thus, to print each module, you need to access each module from the trained HMDT model at first, and then apply the get_rt() function.
"""

from hmmlearn import base_DT, hmm_DT

features = ['Inflow', 'Storage']
smax = 562474

hmdt = pickle_read('path_to_your_HMDT_model')
for index, rt in enumerate(hmdt.model):
    rules = get_rt(rt, smax, features)
    with open(f'./module_{index}.txt', 'w') as f:
        for item in rules:
            f.write(f"{item}\n")
            